In [110]:
import numpy as np
import pandas as pd

In [112]:
# Load CSV
input_csv = "predictions/pm100/predictions.csv"

df = pd.read_csv(input_csv)

In [113]:
df.dtypes

job_id                       int64
user_id                      int64
time_submit                  int64
gt_runtime                   int64
pred_runtime_user            int64
pred_runtime_heuristic     float64
pred_runtime_dt_fs1          int64
pred_runtime_dt_fs2          int64
pred_runtime_dt_fs3          int64
pred_runtime_rnp_fs1         int64
pred_runtime_rnp_fs2         int64
pred_runtime_rnp_fs3         int64
pred_runtime_knn_fs1       float64
pred_runtime_knn_fs2       float64
pred_runtime_knn_fs3       float64
pred_runtime_ora           float64
pred_runtime_knn_c4_fs1        str
pred_runtime_knn_c4_fs2        str
pred_runtime_knn_c7_fs1        str
pred_runtime_knn_c7_fs2        str
dtype: object

In [114]:
df[df.pred_runtime_heuristic > 86400]

,job_id,user_id,time_submit,gt_runtime,pred_runtime_user,pred_runtime_heuristic,pred_runtime_dt_fs1,pred_runtime_dt_fs2,pred_runtime_dt_fs3,pred_runtime_rnp_fs1,pred_runtime_rnp_fs2,pred_runtime_rnp_fs3,pred_runtime_knn_fs1,pred_runtime_knn_fs2,pred_runtime_knn_fs3,pred_runtime_ora,pred_runtime_knn_c4_fs1,pred_runtime_knn_c4_fs2,pred_runtime_knn_c7_fs1,pred_runtime_knn_c7_fs2


In [115]:
df[df.pred_runtime_heuristic > df.pred_runtime_user]

,job_id,user_id,time_submit,gt_runtime,pred_runtime_user,pred_runtime_heuristic,pred_runtime_dt_fs1,pred_runtime_dt_fs2,pred_runtime_dt_fs3,pred_runtime_rnp_fs1,pred_runtime_rnp_fs2,pred_runtime_rnp_fs3,pred_runtime_knn_fs1,pred_runtime_knn_fs2,pred_runtime_knn_fs3,pred_runtime_ora,pred_runtime_knn_c4_fs1,pred_runtime_knn_c4_fs2,pred_runtime_knn_c7_fs1,pred_runtime_knn_c7_fs2
84,4781489,635,11486823,35,2400,3040.830887,4426,4487,4487,4426,86400,86400,196.206674,15688.528986,20502.258610,35.0,Short,Long,Short,Medium-Short
306,2156366,141,11509184,35,3600,4105.033333,387,392,392,387,1,86400,663.365754,578.864046,428.481452,35.0,Short,Short,Medium-Short,Short
313,719710,635,11510589,35,300,3038.913903,387,383,383,387,1,86400,15.250000,4527.764671,3024.982929,35.0,Short,Short,Short,Short
330,3612365,635,11511702,37,300,3035.117197,387,392,392,387,1,86400,15.250000,180.486605,162.626469,37.0,Short,Short,Short,Short
332,5253710,635,11511968,187,1800,3033.208784,387,392,392,387,1,86400,191.001843,341.860496,227.142697,187.0,Short,Short,Short,Short
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66751,3214055,8,13832570,4,300,324.643695,3,2,2,3,3,3,176.716119,72.029198,55.827757,4.0,Short,Very-Short,Short,Very-Short
66752,558069,8,13833191,2,300,324.632916,3,3,3,3,3,3,176.716119,72.031445,55.828004,10.0,Short,Very-Short,Short,Very-Short
66753,1997479,8,13833813,3,300,324.622071,3,2,2,3,3,3,176.716119,72.028828,55.827647,3.0,Short,Very-Short,Short,Very-Short
66754,4424415,8,13834310,3,300,324.611261,3,3,3,3,3,3,176.716119,72.028833,55.827569,300.0,Short,Very-Short,Short,Very-Short


In [116]:
df["prev_pred"] = df.groupby("user_id")["pred_runtime_heuristic"].shift(1)
df["prev_gt"] = df.groupby("user_id")["gt_runtime"].shift(1)

In [117]:
df["pred_ratio"] = df["pred_runtime_heuristic"] / df["prev_pred"]
df["gt_ratio"] = df["prev_gt"] / df["prev_pred"]

In [118]:
PRED_JUMP = 100     # prediction exploded
GT_JUMP   = 250      # real runtime stayed reasonable
MIN_PREV  = 10      # avoid division noise

In [119]:
df["suspicious"] = (
    ((df["pred_ratio"] > PRED_JUMP) & 
    (df["gt_ratio"] <= GT_JUMP))
)

In [120]:
cols = [
    "user_id", "time_submit",
    "prev_pred", "pred_runtime_heuristic", "pred_ratio",
    "prev_gt", "gt_runtime", "gt_ratio"
]

df.loc[df["suspicious"], cols].head(20)

,user_id,time_submit,prev_pred,pred_runtime_heuristic,pred_ratio,prev_gt,gt_runtime,gt_ratio


In [121]:
df.loc[df["suspicious"], "pred_runtime_heuristic"] /= 1000

In [122]:
df

,job_id,user_id,time_submit,gt_runtime,pred_runtime_user,pred_runtime_heuristic,pred_runtime_dt_fs1,pred_runtime_dt_fs2,pred_runtime_dt_fs3,pred_runtime_rnp_fs1,...,pred_runtime_ora,pred_runtime_knn_c4_fs1,pred_runtime_knn_c4_fs2,pred_runtime_knn_c7_fs1,pred_runtime_knn_c7_fs2,prev_pred,prev_gt,pred_ratio,gt_ratio,suspicious
0,5154115,1711,11480812,4578,14400,1582.207613,3488,4571,4449,3722,...,4444.0,Medium,Medium,Medium,Medium,NaN,NaN,NaN,NaN,False
1,5640678,1711,11480812,4614,14400,1581.982636,3488,4454,4652,3722,...,4213.0,Medium,Medium,Medium,Medium,1582.207613,4578.0,0.999858,2.893426,False
2,1597511,1711,11480812,4282,14400,1582.643742,3488,4417,4622,3722,...,4353.0,Medium,Medium,Medium,Medium,1581.982636,4614.0,1.000418,2.916593,False
3,4698474,1711,11480812,4534,14400,1582.843991,3488,4486,4454,3722,...,4353.0,Medium,Medium,Medium,Medium,1582.643742,4282.0,1.000127,2.705599,False
4,6129873,1711,11480812,4140,14400,1581.792801,3488,4564,4454,3722,...,4649.0,Medium,Medium,Medium,Medium,1582.843991,4534.0,0.999336,2.864464,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66751,3214055,8,13832570,4,300,324.643695,3,2,2,3,...,4.0,Short,Very-Short,Short,Very-Short,324.654508,3.0,0.999967,0.009241,False
66752,558069,8,13833191,2,300,324.632916,3,3,3,3,...,10.0,Short,Very-Short,Short,Very-Short,324.643695,4.0,0.999967,0.012321,False
66753,1997479,8,13833813,3,300,324.622071,3,2,2,3,...,3.0,Short,Very-Short,Short,Very-Short,324.632916,2.0,0.999967,0.006161,False
66754,4424415,8,13834310,3,300,324.611261,3,3,3,3,...,300.0,Short,Very-Short,Short,Very-Short,324.622071,3.0,0.999967,0.009242,False


In [123]:
df.drop(columns=["prev_pred"], inplace=True)
df.drop(columns=["prev_gt"], inplace=True)
df.drop(columns=["pred_ratio"], inplace=True)
df.drop(columns=["gt_ratio"], inplace=True)
df.drop(columns=["suspicious"], inplace=True)

In [124]:
df

,job_id,user_id,time_submit,gt_runtime,pred_runtime_user,pred_runtime_heuristic,pred_runtime_dt_fs1,pred_runtime_dt_fs2,pred_runtime_dt_fs3,pred_runtime_rnp_fs1,pred_runtime_rnp_fs2,pred_runtime_rnp_fs3,pred_runtime_knn_fs1,pred_runtime_knn_fs2,pred_runtime_knn_fs3,pred_runtime_ora,pred_runtime_knn_c4_fs1,pred_runtime_knn_c4_fs2,pred_runtime_knn_c7_fs1,pred_runtime_knn_c7_fs2
0,5154115,1711,11480812,4578,14400,1582.207613,3488,4571,4449,3722,3062,3114,3329.945545,4526.656984,4535.613007,4444.0,Medium,Medium,Medium,Medium
1,5640678,1711,11480812,4614,14400,1581.982636,3488,4454,4652,3722,3029,3081,3329.945545,4503.408689,4504.381573,4213.0,Medium,Medium,Medium,Medium
2,1597511,1711,11480812,4282,14400,1582.643742,3488,4417,4622,3722,3138,3190,3329.945545,4477.197732,4486.737842,4353.0,Medium,Medium,Medium,Medium
3,4698474,1711,11480812,4534,14400,1582.843991,3488,4486,4454,3722,3092,3144,3329.945545,4502.364515,4500.269448,4353.0,Medium,Medium,Medium,Medium
4,6129873,1711,11480812,4140,14400,1581.792801,3488,4564,4454,3722,3088,3140,3329.945545,4505.689422,4518.305529,4649.0,Medium,Medium,Medium,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66751,3214055,8,13832570,4,300,324.643695,3,2,2,3,3,3,176.716119,72.029198,55.827757,4.0,Short,Very-Short,Short,Very-Short
66752,558069,8,13833191,2,300,324.632916,3,3,3,3,3,3,176.716119,72.031445,55.828004,10.0,Short,Very-Short,Short,Very-Short
66753,1997479,8,13833813,3,300,324.622071,3,2,2,3,3,3,176.716119,72.028828,55.827647,3.0,Short,Very-Short,Short,Very-Short
66754,4424415,8,13834310,3,300,324.611261,3,3,3,3,3,3,176.716119,72.028833,55.827569,300.0,Short,Very-Short,Short,Very-Short


In [125]:
# Convert to integer
df["pred_runtime_heuristic"] = (
    df["pred_runtime_heuristic"]
    .round()
    .astype(int)
)

df["pred_runtime_dt_fs1"] = (
    df["pred_runtime_dt_fs1"]
    .round()
    .astype(int)
)

df["pred_runtime_dt_fs2"] = (
    df["pred_runtime_dt_fs2"]
    .round()
    .astype(int)
)

df["pred_runtime_rnp_fs1"] = (
    df["pred_runtime_rnp_fs1"]
    .round()
    .astype(int)
)

df["pred_runtime_rnp_fs2"] = (
    df["pred_runtime_rnp_fs2"]
    .round()
    .astype(int)
)

df["pred_runtime_knn_fs1"] = (
    df["pred_runtime_knn_fs1"]
    .round()
    .astype(int)
)

df["pred_runtime_knn_fs2"] = (
    df["pred_runtime_knn_fs2"]
    .round()
    .astype(int)
)

df["pred_runtime_ora"] = (
    df["pred_runtime_ora"]
    .round()
    .astype(int)
)

In [126]:
# Cap predictions to user prediction
df["pred_runtime_heuristic"] = np.minimum(
    df["pred_runtime_heuristic"],
    df["pred_runtime_user"]
)

df["pred_runtime_dt_fs1"] = np.minimum(
    df["pred_runtime_dt_fs1"],
    df["pred_runtime_user"]
)

df["pred_runtime_dt_fs2"] = np.minimum(
    df["pred_runtime_dt_fs2"],
    df["pred_runtime_user"]
)

df["pred_runtime_rnp_fs1"] = np.minimum(
    df["pred_runtime_rnp_fs1"],
    df["pred_runtime_user"]
)

df["pred_runtime_rnp_fs2"] = np.minimum(
    df["pred_runtime_rnp_fs2"],
    df["pred_runtime_user"]
)

df["pred_runtime_knn_fs1"] = np.minimum(
    df["pred_runtime_knn_fs1"],
    df["pred_runtime_user"]
)

df["pred_runtime_knn_fs2"] = np.minimum(
    df["pred_runtime_knn_fs2"],
    df["pred_runtime_user"]
)

df["pred_runtime_ora"] = np.minimum(
    df["pred_runtime_ora"],
    df["pred_runtime_user"]
)

In [127]:
df

,job_id,user_id,time_submit,gt_runtime,pred_runtime_user,pred_runtime_heuristic,pred_runtime_dt_fs1,pred_runtime_dt_fs2,pred_runtime_dt_fs3,pred_runtime_rnp_fs1,pred_runtime_rnp_fs2,pred_runtime_rnp_fs3,pred_runtime_knn_fs1,pred_runtime_knn_fs2,pred_runtime_knn_fs3,pred_runtime_ora,pred_runtime_knn_c4_fs1,pred_runtime_knn_c4_fs2,pred_runtime_knn_c7_fs1,pred_runtime_knn_c7_fs2
0,5154115,1711,11480812,4578,14400,1582,3488,4571,4449,3722,3062,3114,3330,4527,4535.613007,4444,Medium,Medium,Medium,Medium
1,5640678,1711,11480812,4614,14400,1582,3488,4454,4652,3722,3029,3081,3330,4503,4504.381573,4213,Medium,Medium,Medium,Medium
2,1597511,1711,11480812,4282,14400,1583,3488,4417,4622,3722,3138,3190,3330,4477,4486.737842,4353,Medium,Medium,Medium,Medium
3,4698474,1711,11480812,4534,14400,1583,3488,4486,4454,3722,3092,3144,3330,4502,4500.269448,4353,Medium,Medium,Medium,Medium
4,6129873,1711,11480812,4140,14400,1582,3488,4564,4454,3722,3088,3140,3330,4506,4518.305529,4649,Medium,Medium,Medium,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66751,3214055,8,13832570,4,300,300,3,2,2,3,3,3,177,72,55.827757,4,Short,Very-Short,Short,Very-Short
66752,558069,8,13833191,2,300,300,3,3,3,3,3,3,177,72,55.828004,10,Short,Very-Short,Short,Very-Short
66753,1997479,8,13833813,3,300,300,3,2,2,3,3,3,177,72,55.827647,3,Short,Very-Short,Short,Very-Short
66754,4424415,8,13834310,3,300,300,3,3,3,3,3,3,177,72,55.827569,300,Short,Very-Short,Short,Very-Short


In [128]:
# Save corrected CSV
df.to_csv(input_csv, index=False)

print(f"Corrected file saved to: {input_csv}")

Corrected file saved to: predictions/pm100/predictions.csv
